# ANALYSIS

In [1]:
#uncomment this below code to install imblearn package
# !pip install imbalanced-learn

In [2]:
# !pip install scikit-optimize

In [3]:
import pandas as pd
import numpy as np
import sklearn

#statistics
from scipy.stats import chi2_contingency, ttest_ind

import cudf #gpu-powered DataFrame (Pandas alternative)

#imbalance handling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, RepeatedEditedNearestNeighbours
from imblearn.pipeline import Pipeline

#preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler

#hyperparameter search
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize

#internal validation
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedStratifiedKFold, cross_val_score, GridSearchCV, PredefinedSplit, RandomizedSearchCV


#performance metrices
from sklearn.metrics import make_scorer, confusion_matrix, classification_report, f1_score, balanced_accuracy_score, r2_score, auc, average_precision_score, roc_auc_score, recall_score, roc_curve, accuracy_score

#Models selection
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from cuml.svm import SVC #gpu-powered SVM


#save and load trained model
import pickle

#visualisation
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

import os

In [4]:
# Data loader
features = pd.read_csv("../FinalData/cleaned_features_11072023.csv")
outcomes = pd.read_csv("../FinalData/cleaned_outcomes_11072023.csv")
features = features[features.columns[1:]]
# outcomes = outcomes[outcomes.columns[1:]]

/tmp/ipykernel_12328/3164845275.py:2: DtypeWarning: Columns (26,27,29,30,33,34,68,72) have mixed types. Specify dtype option on import or set low_memory=False.
  features = pd.read_csv("../FinalData/cleaned_features_11072023.csv")


In [5]:
print(features.shape)
print(outcomes.shape)

(577962, 87)
(577962, 15)


In [6]:
features.columns

Index(['patid', 'practice_id', 'sex', 'age', 'BMI', 'weight', 'height',
       'ethnicity', 'ethnic_group', 'smokingStatus', 'CharlsonScore',
       'count_rhinitis', 'count_cardiovascular', 'count_heartfailure',
       'count_psoriasis', 'count_anaphylaxis', 'count_diabetes', 'count_ihd',
       'count_anxiety', 'count_eczema', 'count_nasalpolyps',
       'count_paracetamol', 'count_nsaids', 'count_betablocker', 'id',
       'event_date', 'recorded_date', 'visit_id', 'code_id', 'snomed_id',
       'numeric_1', 'numeric_2', 'created_datetime', 'updated_datetime',
       'PEFStatus', 'EosinophilLevel', 'BTS_step', 'average_daily_dose_ICS',
       'prescribed_daily_dose_ICS', 'ICS_medication_possesion_ratio',
       'DeviceType', 'Spacer', 'numOCS', 'PriorEducation', 'numPCS',
       'numPCSAsthma', 'numAntibioticsEvents', 'numAntibioticswithLRTI',
       'numOCSEvents', 'numOCSwithLRTI', 'numAsthmaAttacks',
       'numAcuteRespEvents', 'numHospEvents', 'postcode_district',
       'imd_d

In [7]:
features.head()

,patid,practice_id,sex,age,BMI,weight,height,ethnicity,ethnic_group,smokingStatus,...,psoriasis,anaphylaxis,diabetes,ihd,anxiety,eczema,nasalpolyps,paracetamol,nsaids,betablocker
0,43231452,39,0.0,48,26.609713,76.0,1.6900,not_recorded,not_recorded,Active Smoker,...,0,0,0,0,0,0,0,0,0,0
1,43206365,39,1.0,58,23.946360,72.5,1.7400,not_recorded,not_recorded,Former Smoker,...,0,0,0,0,1,0,0,0,0,0
2,43203606,559,0.0,51,17.104513,39.0,1.5100,not_recorded,not_recorded,Active Smoker,...,0,0,0,0,0,1,0,0,0,0
3,43117348,502,0.0,69,35.303241,74.0,1.4478,not_recorded,not_recorded,Former Smoker,...,1,0,0,0,0,1,0,0,0,0
4,43289035,593,0.0,49,31.217482,75.0,1.5500,White British,White - ethnic group,Active Smoker,...,0,0,0,0,0,1,0,0,0,0


In [8]:
outcomes.head()

,patid,outcome_3months,outcome_6months,outcome_9months,outcome_12months,outcome_15months,outcome_18months,outcome_21months,outcome_24months,outcome_combined_6months,outcome_combined_9months,outcome_combined_12months,outcome_combined_15months,outcome_combined_18months,outcome_combined_24months
0,43231452,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,43206365,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,43203606,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,43117348,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,43289035,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
masterData = features.merge(outcomes, how = 'left', left_on='patid', right_on='patid') #join table
# masterData = masterData.dropna() #NAs from Country
masterData = masterData.reset_index(drop=True)
exclude_columns = ['weight', 'height', 'id', 'event_date', 'recorded_date', 'visit_id', 'code_id', 'snomed_id',
       'numeric_1', 'numeric_2', 'created_datetime', 'updated_datetime',]
masterData = masterData.loc[:,~masterData.columns.isin(exclude_columns)]
print('original data shape: ', masterData.shape)

original data shape:  (577962, 89)


In [10]:
pd.options.display.max_rows = 287
masterData.isna().sum()

patid                                 0
practice_id                           0
sex                                   0
age                                   0
BMI                                   0
ethnicity                             0
ethnic_group                          0
smokingStatus                         0
CharlsonScore                         0
count_rhinitis                        0
count_cardiovascular                  0
count_heartfailure                    0
count_psoriasis                       0
count_anaphylaxis                     0
count_diabetes                        0
count_ihd                             0
count_anxiety                         0
count_eczema                          0
count_nasalpolyps                     0
count_paracetamol                     0
count_nsaids                          0
count_betablocker                     0
PEFStatus                             0
EosinophilLevel                       0
BTS_step                              0


In [11]:
#Positive vs negative class ratio

print('3 months -> 1 : ', round(masterData.outcome_3months.value_counts()[0]/masterData.outcome_3months.value_counts()[1],2))
print('6 months -> 1 : ', round(masterData.outcome_combined_6months.value_counts()[0]/masterData.outcome_combined_6months.value_counts()[1],2))
print('9 months -> 1 : ', round(masterData.outcome_combined_9months.value_counts()[0]/masterData.outcome_combined_9months.value_counts()[1],2))
print('12 months -> 1 : ', round(masterData.outcome_combined_12months.value_counts()[0]/masterData.outcome_combined_12months.value_counts()[1],2))
print('15 months -> 1 : ', round(masterData.outcome_combined_15months.value_counts()[0]/masterData.outcome_combined_15months.value_counts()[1],2))
print('18 months -> 1 : ', round(masterData.outcome_combined_18months.value_counts()[0]/masterData.outcome_combined_18months.value_counts()[1],2))
print('24 months -> 1 : ', round(masterData.outcome_combined_24months.value_counts()[0]/masterData.outcome_combined_24months.value_counts()[1],2))


3 months -> 1 :  35.63
6 months -> 1 :  23.46
9 months -> 1 :  18.06
12 months -> 1 :  13.49
15 months -> 1 :  10.92
18 months -> 1 :  9.91
24 months -> 1 :  8.85


In [12]:
#Proportion of asthma attack in each outcome

print('3 months -> ', round(masterData.outcome_3months.value_counts()[1]/len(masterData)*100,2), '%')
print('6 months -> ', round(masterData.outcome_combined_6months.value_counts()[1]/len(masterData)*100,2), '%')
print('9 months -> ', round(masterData.outcome_combined_9months.value_counts()[1]/len(masterData)*100,2), '%')
print('12 months -> ', round(masterData.outcome_combined_12months.value_counts()[1]/len(masterData)*100,2), '%')
print('15 months -> ', round(masterData.outcome_combined_15months.value_counts()[1]/len(masterData)*100,2), '%')
print('18 months -> ', round(masterData.outcome_combined_18months.value_counts()[1]/len(masterData)*100,2), '%')
print('24 months -> ', round(masterData.outcome_combined_24months.value_counts()[1]/len(masterData)*100,2), '%')

3 months ->  2.73 %
6 months ->  4.09 %
9 months ->  5.25 %
12 months ->  6.9 %
15 months ->  8.39 %
18 months ->  9.16 %
24 months ->  10.15 %


In [13]:
#Data scenario
# 1: all data without ethnicity variable
# 2: all data with ethnicity variable (include all missing values in ethnicity as separate group)
# 3: filter data based on ethnicity (exclude missing values)

scenario = 1 #change it based on the scenario

if scenario == 1:
    #include all data
    allData = masterData
    
elif scenario == 2:
    #Exclude ethnic column
    allData = masterData.drop('ethnic_group', axis=1)
    
elif scenario == 3:
    #exclude missing values for ethnic variable
    allData = masterData[masterData.ethnic_group!='not_recorded']
    
allData = allData.reset_index(drop=True)
print('Data shape for scenario', str(scenario), allData.shape)



Data shape for scenario 1 (577962, 89)


In [14]:
#Split data into training and evaluation set based on the country. Include only 18+ patients.

trainingData = allData[(allData.Country == 'England') & (allData.age>=18)]
evaluationData = allData[((allData.Country == 'Scotland') | (allData.Country == 'Wales')) & (allData.age>=18)] #used for internal validation

#remove country variable
trainingData = trainingData.drop('Country', axis=1)
evaluationData = evaluationData.drop('Country', axis=1)

trainingData = trainingData.reset_index(drop=True)
evaluationData = evaluationData.reset_index(drop=True)

print('Training data shape:', trainingData.shape)
print('Evaluation data shape: ', evaluationData.shape)

Training data shape: (555836, 88)
Evaluation data shape:  (22126, 88)


In [15]:
#encode categorical data

cat_vars = ['PEFStatus','EosinophilLevel']
onehot_vars = ['ethnic_group','smokingStatus', 'DeviceType', 'cat_BMI', 'imd_decile']
data_categorical = trainingData[cat_vars]
data_onehot = trainingData[onehot_vars]

#ordinal encoder
encoder = OrdinalEncoder(categories=[['not_recorded','less than 60', '60-80', 'more than 80'], ['unknown', 'normal', 'high']]).set_output(transform="pandas")
data_encoded = encoder.fit_transform(data_categorical)
pickle.dump(encoder, open('../Models/cat_encoder.pkl', 'wb'))
    
#one hot encoder
onehot_encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
onehot_encoded = onehot_encoder.fit_transform(data_onehot)
pickle.dump(onehot_encoder, open('../Models/onehot_encoder.pkl', 'wb'))

trainingData = pd.concat([trainingData.drop(cat_vars, axis=1), data_encoded], axis=1)
trainingData = pd.concat([trainingData.drop(onehot_vars, axis=1), onehot_encoded], axis=1)

print('Data shape after encoding: ', trainingData.shape)

Data shape after encoding:  (555836, 113)


In [16]:
#Data normalisation for continous variable into 0-1 range


continuous_vars = ['age', 'CharlsonScore', 'average_daily_dose_ICS', 'prescribed_daily_dose_ICS', 'ICS_medication_possesion_ratio', 
                   'numOCS', 'numPCS', 'numPCSAsthma', 'numAntibioticsEvents', 'numAntibioticswithLRTI', 'numOCSEvents', 'numOCSwithLRTI', 
                   'numAsthmaAttacks', 'numAcuteRespEvents', 'numHospEvents']

# define scaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(trainingData[continuous_vars])
pickle.dump(scaler, open('../Models/cont_scaler.pkl', 'wb'))


data_scaled = pd.DataFrame(data_scaled, columns=scaler.get_feature_names_out())
trainingData = pd.concat([trainingData.drop(continuous_vars, axis=1), data_scaled], axis=1)

print('Data shape after scaling: ', trainingData.shape)

Data shape after scaling:  (555836, 113)


In [17]:
#Define feature candidates

features_columns = trainingData.columns.to_list()
exclude_columns = ['patid', 'practice_id', #identifier
                   'BMI', #use the categorical instead
                   'ethnicity', #use ethnic_group instead
                   'Spacer',  #all zero
                   
                   'outcome_3months', 'outcome_6months', 'outcome_9months', 'outcome_12months', 'outcome_15months', 'outcome_18months', 
                   'outcome_21months', 'outcome_24months', 'outcome_combined_6months', 'outcome_combined_9months', 'outcome_combined_12months', 
                   'outcome_combined_15months', 'outcome_combined_18months', 'outcome_combined_24months', #outcomes variable
                   
                   'postcode_district', 'County', 'LocalAuthority', 'OutputAreaClassification', #location related variables, use IMD decile only
                   
                   'cat_age', 'cat_average_daily_dose_ICS', 'cat_prescribed_daily_dose_ICS', 'cat_ICS_medication_possesion_ratio', 'cat_numOCS', 'cat_numOCSEvents', 
                   'cat_numOCSwithLRTI', 'cat_numAcuteRespEvents', 'cat_numAntibioticsEvents', 'cat_numAntibioticswithLRTI', 'cat_numAsthmaAttacks', 'cat_numHospEvents', 
                   'cat_numPCS', 'cat_numPCSAsthma', #use continous vars instead
                   
                   'count_rhinitis', 'count_cardiovascular', 'count_heartfailure',
                   'count_psoriasis', 'count_anaphylaxis', 'count_diabetes', 'count_ihd',
                   'count_anxiety', 'count_eczema', 'count_nasalpolyps',
                   'count_paracetamol', 'count_nsaids', 'count_betablocker', #use binary ones
                  ]
exclude_columns = exclude_columns + [x for x in features_columns if '_count' in x] #filter out commorbid count variables
features_columns = [x for x in features_columns if x not in exclude_columns]
print('Features size: ', len(features_columns))
print(features_columns)

Features size:  63
['sex', 'BTS_step', 'PriorEducation', 'rhinitis', 'cardiovascular', 'heartfailure', 'psoriasis', 'anaphylaxis', 'diabetes', 'ihd', 'anxiety', 'eczema', 'nasalpolyps', 'paracetamol', 'nsaids', 'betablocker', 'PEFStatus', 'EosinophilLevel', 'ethnic_group_Asian - ethnic group', 'ethnic_group_Black - ethnic group', 'ethnic_group_Mixed ethnic census group', 'ethnic_group_Other ethnic group', 'ethnic_group_White - ethnic group', 'ethnic_group_not_recorded', 'smokingStatus_Active Smoker', 'smokingStatus_Former Smoker', 'smokingStatus_Non Smoker', 'DeviceType_BAI', 'DeviceType_DPI', 'DeviceType_NEB', 'DeviceType_pMDI', 'DeviceType_unknown', 'cat_BMI_normal', 'cat_BMI_not recorded', 'cat_BMI_obese', 'cat_BMI_overweight', 'cat_BMI_underweight', 'imd_decile_0', 'imd_decile_1', 'imd_decile_2', 'imd_decile_3', 'imd_decile_4', 'imd_decile_5', 'imd_decile_6', 'imd_decile_7', 'imd_decile_8', 'imd_decile_9', 'imd_decile_10', 'age', 'CharlsonScore', 'average_daily_dose_ICS', 'prescrib

In [ ]:
#ONE HOT encoding for categorical data

# categoricalNonnumericVars = pd.Series(list(set(categoricalNonnumericVars).intersection(set(features_columns)))).tolist() #select only variables within the feature candidate list

# # define one hot encoder
# categoricalEncoder = OneHotEncoder(sparse=False)

# # transform data
# result = categoricalEncoder.fit_transform(trainingData[categoricalNonnumericVars]) 
# result = pd.DataFrame(result, columns=categoricalEncoder.get_feature_names_out())

# #save encoder
# pickle.dump(categoricalEncoder, open('./models/categoricalEncoder.pkl', 'wb'))

# # replace categorical variables in the original data with the one hot version
# trainingData = pd.concat([trainingData.loc[:, ~trainingData.columns.isin(categoricalNonnumericVars)],result], axis=1)
# print('Data shape after one-hot encoding: ', trainingData.shape)

In [ ]:
# #ONE HOT encoding for evaluation dataset

# # transform data
# result = categoricalEncoder.transform(evaluationData[categoricalNonnumericVars]) 
# result = pd.DataFrame(result, columns=categoricalEncoder.get_feature_names_out())

# # replace categorical variables in the original data with the one hot version
# evaluationData = pd.concat([evaluationData.loc[:, ~evaluationData.columns.isin(categoricalNonnumericVars)],result], axis=1)
# print('Data shape after one-hot encoding: ', evaluationData.shape)

In [ ]:
# #Scaling continous variable into 0-1 range for evaluation dataset


# # transform data
# result = scaler.transform(evaluationData[continuous_vars])
# result = pd.DataFrame(result, columns=scaler.get_feature_names_out())

# evaluationData = pd.concat([evaluationData.loc[:,~evaluationData.columns.isin(continuous_vars)],result], axis=1)

# print('Data shape after scaling: ', evaluationData.shape)

In [ ]:
# #Model evaluation function

# def summariseResult (testX, testY, model):
#     preds = model.predict(testX)
#     tn, fp, fn, tp = confusion_matrix(testY, preds).ravel()
#     specificity = tn / (tn+fp)
#     sensitivity = tp / (tp+fn)
#     ppv = 100*tp/(tp+fp)
#     npv = 100*tn/(fn+tn)
#     acc = accuracy_score(testY, preds)
#     f1score = f1_score(testY, preds, average = 'binary')
#     balanceacc = balanced_accuracy_score(testY, preds)
#     fpr, tpr, thresholds = roc_curve(testY, preds, pos_label=1)
#     aucscore = auc(fpr, tpr)
#     # auc = roc_auc_score(testY, preds)
#     auprc = average_precision_score(testY, preds)
#     # plot_confusion_matrix(model, testX, testY, cmap='viridis')  
#     return np.round(acc,4), np.round(specificity,4), np.round(sensitivity,4), np.round(aucscore,4), np.round(auprc,4), np.round(balanceacc,4), np.round(f1score,4), np.round(ppv,4), np.round(npv,4)

In [ ]:
# #Fix model name for visualisation

# def modelNameFixer(x):
#     if 'liblinear' in x:
#         return 'Lasso'
#     elif 'GaussianNB' in x:
#         return 'GNB'
#     elif 'SVC' in x:
#         return 'SVC'
#     elif 'RandomForest' in x:
#         return 'RF'
#     elif 'XGB' in x:
#         return 'XGBoost'
#     elif 'DecisionTree' in x:
#         return 'DT'
#     else:
#         return 'LR'

In [ ]:
# # instantiate the model (using the default parameters)
# def build_models (X_train, y_train, params, split_counter):
#     models = [] #list to store all the models
#     model_counter = 0
#     print("Building models . . . .")

#     #LR
#     lr_model = LogisticRegression(class_weight='balanced', penalty='l2', random_state=1234)
#     lr_model.fit(X_train,y_train)
#     modelname =str(split_counter) + 'LRModel' 
#     models.append([modelname, y_train.value_counts()[1]/y_train.value_counts()[0]])
#     model_counter+=1
#     pickle.dump(lr_model, open('./models/'+ target_outcome + '/'+ modelname + '.sav', 'wb')) 
#     print("LR done")

#     #Lasso
#     lasso_model = LogisticRegression(class_weight='balanced', penalty='l1', solver='liblinear', random_state=1234) #only the LIBLINEAR and SAGA (added in v0.19) solvers handle the L1 penalty
#     lasso_model.fit(X_train, y_train)
#     modelname =str(split_counter) + 'LassoModel' 
#     models.append([modelname, y_train.value_counts()[1]/y_train.value_counts()[0]])
#     model_counter+=1
#     pickle.dump(lasso_model, open('./models/'+ target_outcome + '/'+ modelname + '.sav', 'wb'))
#     print("LR done")

#     #GNB
#     gnb_model = GaussianNB()
#     gnb_model.fit(X_train, y_train)
#     modelname =str(split_counter) + 'GNBModel' 
#     models.append([modelname, y_train.value_counts()[1]/y_train.value_counts()[0]])
#     model_counter+=1
#     pickle.dump(gnb_model, open('./models/'+ target_outcome + '/'+ modelname + '.sav', 'wb'))     
#     print("GNB done")

#     #SVM
#     svc_model = SVC(class_weight='balanced', C = 0.7, degree=2, kernel='poly', random_state=1234, cache_size=2048)
#     svc_model.fit(X_train,y_train)
#     modelname =str(split_counter) + 'SVCModel' 
#     models.append([modelname, y_train.value_counts()[1]/y_train.value_counts()[0]])
#     model_counter+=1
#     pickle.dump(svc_model, open('./models/'+ target_outcome + '/'+ modelname + '.sav', 'wb'))     
#     print("SVM done")

#     #DT
#     dt_model = DecisionTreeClassifier(class_weight='balanced', random_state=1234)
#     dt_model.fit(X_train, y_train)
#     modelname =str(split_counter) + 'DTModel' 
#     models.append([modelname, y_train.value_counts()[1]/y_train.value_counts()[0]])
#     model_counter+=1
#     pickle.dump(dt_model, open('./models/'+ target_outcome + '/'+ modelname + '.sav', 'wb'))     
#     print("DT done")

#     #RF
#     rf_model = RandomForestClassifier(class_weight='balanced', n_estimators=500, random_state=1234)
#     rf_model.fit(X_train, y_train)
#     modelname =str(split_counter) + 'RFModel' 
#     models.append([modelname, y_train.value_counts()[1]/y_train.value_counts()[0]])
#     model_counter+=1
#     pickle.dump(rf_model, open('./models/'+ target_outcome + '/'+ modelname + '.sav', 'wb'))     
#     print("RF done")



#     #XGB
#     scale_pos_ratio = y_train.value_counts()[0]/y_train.value_counts()[1]
#     xgb_model = xgb.XGBClassifier(objective ='binary:logistic', max_depth = params['xgb_maxdepth'], n_estimators = 2000,  tree_method='gpu_hist', gpu_id=0,  verbosity = 0, random_state = 1234,
#                                  importance_type = 'gain', scale_pos_weight = scale_pos_ratio, use_label_encoder=False, learning_rate=params['xgb_lr'])
#     # xgb_model = xgb.XGBClassifier(objective ='binary:logistic', learning_rate = 0.001, tree_method='gpu_hist', gpu_id=0,  verbosity = 0, random_state = 1234)
#     xgb_model.fit(X_train,y_train)
#     #save model
#     modelname = str(split_counter) + 'XGBoostModel'
#     models.append([modelname,  y_train.value_counts()[1]/y_train.value_counts()[0]])
#     pickle.dump(xgb_model, open('./models/'+ target_outcome + '/'+ modelname + '.sav', 'wb')) 
#     model_counter+=1
#     print("XGB done")
    
#     return models
#     # return [xgb_model]

In [ ]:
outcomes.columns

# GRID SEARCH

In [18]:
def print_dataframe(filtered_cv_results):
    """Pretty print for filtered dataframe"""
    for mean_sensitivity, std_sensitivity, mean_specificity, std_specificity, mean_auc, std_auc, params in zip(
        filtered_cv_results["mean_test_sensitivity"],
        filtered_cv_results["std_test_sensitivity"],
        filtered_cv_results["mean_test_specificity"],
        filtered_cv_results["std_test_specificity"],
        filtered_cv_results["mean_test_auc"],
        filtered_cv_results["std_test_auc"],
        filtered_cv_results["params"],
    ):
        print(
            f"sensitivity: {mean_sensitivity:0.4f} (±{std_sensitivity:0.03f}),"
            f" specificity: {mean_specificity:0.4f} (±{std_specificity:0.03f}),"
            f"auc: {mean_auc:0.4f} (±{std_auc:0.03f}),"
            f" for {params}"
        )
    print()


def refit_strategy(cv_results):
    """Define the strategy to select the best estimator.

    The strategy defined here is to filter-out all results below a precision threshold
    of 0.98, rank the remaining by recall and keep all models with one standard
    deviation of the best by recall. Once these models are selected, we can select the
    fastest model to predict.

    Parameters
    ----------
    cv_results : dict of numpy (masked) ndarrays
        CV results as returned by the `GridSearchCV`.

    Returns
    -------
    best_index : int
        The index of the best estimator as it appears in `cv_results`.
    """
    # print the info about the grid-search for the different scores
    sensitivity_threshold = 0.5

    cv_results_ = pd.DataFrame(cv_results)
    print("All grid-search results:")
    print_dataframe(cv_results_)

    # Filter-out all results below the threshold
    high_sensitivity_cv_results = cv_results_[
        cv_results_["mean_test_sensitivity"] > sensitivity_threshold
    ]

    print(f"Models with a sensitivity higher than {sensitivity_threshold}:")
    print_dataframe(high_sensitivity_cv_results)

    high_sensitivity_cv_results = high_sensitivity_cv_results[
        [
            "mean_score_time",
            "mean_test_sensitivity",
            "std_test_sensitivity",
            "mean_test_specificity",
            "std_test_specificity",
            "mean_test_auc",
            "std_test_auc",
            "rank_test_sensitivity",
            "rank_test_specificity",
            "rank_test_auc",
            "params",
        ]
    ]

    # Select the most performant models in terms of sesntivity
    # (within 1 sigma from the best)
    best_auc_std = high_sensitivity_cv_results["mean_test_auc"].std()
    best_auc = high_sensitivity_cv_results["mean_test_auc"].max()
    best_auc_threshold = best_auc - best_auc_std

    high_auc_cv_results = high_sensitivity_cv_results[
        high_sensitivity_cv_results["mean_test_auc"] > best_auc_threshold
    ]
    if high_auc_cv_results.shape[0] > 1:
        print(
            "Out of the previously selected high sensitivity models, we keep all the\n"
            "the models within one standard deviation of the highest auc model:"
        )

        print(best_auc_threshold)
        print_dataframe(high_auc_cv_results)

        # From the best candidates, select the fastest model to predict
        fastest_top_auc_high_sensitivity_index = high_auc_cv_results[
            "mean_score_time"
        ].idxmin()

        print(
            "\nThe selected final model is the fastest to predict out of the previously\n"
            "selected subset of best models based on precision and recall.\n"
            "Its scoring time is:\n\n"
            f"{high_auc_cv_results.loc[fastest_top_auc_high_sensitivity_index]}"
        )

        return fastest_top_auc_high_sensitivity_index
    elif high_auc_cv_results.shape[0] == 1:
        print('no parameter achieve the threshold, so return the default best score')
        return cv_results_["mean_test_auc"].idxmax()
    else:
        print('no parameter achieve the threshold, so return the default best score')
        return cv_results_["mean_test_auc"].idxmax()

In [20]:
# LR = ['solver', 'C', 'max_iter']
# Lasso = ['solver', 'C', 'max_iter']
# Elastic= ['l1_ratio', 'max_iter']
# GNB = ['var_smoothing']
# SVM = ['C', 'gamma']
# DT = ['criterion', 'splitter', 'max_depth']
# RF = ['criterion', 'n_estimators', 'max_depth']
# XGB = ['n_estimators', 'learning_rate', 'reg_alpha', 'reg_lambda']


In [19]:
%%time

#Bayesiain continuous search

X = trainingData[features_columns]
outcomes = [
            # 'outcome_3months', 
            'outcome_combined_6months', 
            # 'outcome_combined_12months', 
            # 'outcome_combined_24months',
           ] 
cv = StratifiedKFold(n_splits=3)
n_calls = 10
n_jobs = 2

output = []

for target_outcome in outcomes:
    print('######################################################################################################')
    print(target_outcome)
    y = trainingData[target_outcome]
    scale_pos_ratio = y.value_counts()[0]/y.value_counts()[1]
    
#     if target_outcome == 'outcome_combined_24months':   
#     ##############################################################################
#         print('#LR')
#         lr_model = LogisticRegression(class_weight='balanced', random_state=1234)
#         lr_params = [Categorical(['liblinear', 'newton-cholesky'], name = 'solver'),
#                      Real(0.1, 10, 'log-uniform', name='C'), 
#                      Integer(50, 200, 'uniform', name='max_iter')]

#         @use_named_args(lr_params)
#         def lr_objective(**params):
#             lr_model.set_params(**params)

#             return 1-np.mean(cross_val_score(lr_model, X, y, cv=cv,
#                                             scoring=make_scorer(roc_auc_score)))

#         res_gp_lr = gp_minimize(lr_objective, lr_params, n_calls=n_calls, random_state=1234, verbose=3, n_jobs=n_jobs)
#         output.append([target_outcome, 'lr', 1-res_gp_lr.fun, res_gp_lr.x])

#     ########################################################################################

#         print('#Lasso')
#         lasso_model = LogisticRegression(class_weight='balanced', penalty='l1', random_state=1234) #only the LIBLINEAR and SAGA (added in v0.19) solvers handle the L1 penalty
#         lasso_params = [Categorical(['saga', 'liblinear'], name = 'solver'),
#                           Real(0.1, 10, 'log-uniform', name='C'),
#                           Integer(50, 200, 'uniform', name='max_iter')]

#         @use_named_args(lasso_params)
#         def lasso_objective(**params):
#             lasso_model.set_params(**params)

#             return 1-np.mean(cross_val_score(lasso_model, X, y, cv=cv,
#                                             scoring=make_scorer(roc_auc_score)))

#         res_gp_lasso = gp_minimize(lasso_objective, lasso_params, n_calls=n_calls, random_state=1234, verbose=3, n_jobs=n_jobs)
#         output.append([target_outcome, 'lasso', 1-res_gp_lasso.fun, res_gp_lasso.x])

#     ########################################################################################

#         print('#Elastic')
#         elastic_model = LogisticRegression(class_weight='balanced', penalty = 'elasticnet', solver = 'saga', random_state=1234)
#         elastic_params = [Real(0.1, 1, 'log-uniform', name='l1_ratio'),
#                           Integer(300, 800, 'uniform', name='max_iter')]

#         @use_named_args(elastic_params)
#         def elastic_objective(**params):
#             elastic_model.set_params(**params)

#             return 1-np.mean(cross_val_score(elastic_model, X, y, cv=cv,
#                                             scoring=make_scorer(roc_auc_score)))

#         res_gp_elastic = gp_minimize(elastic_objective, elastic_params, n_calls=n_calls, random_state=1234, verbose=3, n_jobs=n_jobs)
#         output.append([target_outcome, 'elastic', 1-res_gp_elastic.fun, res_gp_elastic.x])

#     ########################################################################################

#         print('#NB')
#         gnb_model = GaussianNB()
#         gnb_params = [Real(1e-9, 1e-5, 'log-uniform', name='var_smoothing')]

#         @use_named_args(gnb_params)
#         def gnb_objective(**params):
#             gnb_model.set_params(**params)

#             return 1-np.mean(cross_val_score(gnb_model, X, y, cv=cv,
#                                             scoring = make_scorer(roc_auc_score)))

#         res_gp_gnb = gp_minimize(gnb_objective, gnb_params, n_calls=n_calls, random_state=1234, verbose=3, n_jobs=n_jobs)
#         output.append([target_outcome, 'gnb', 1-res_gp_gnb.fun, res_gp_gnb.x])

    ########################################################################################

    print('#SVM')
    svc_model = SVC(class_weight='balanced', kernel='rbf', cache_size=1000, random_state=1234)
    svm_params = [Real(0.1, 100, "log-uniform", name='C'),
                     Real(0.1, 100, "log-uniform", name='gamma')]

    @use_named_args(svm_params)
    def svm_objective(**params):
        svc_model.set_params(**params)

        return 1-np.mean(cross_val_score(svc_model, X, y, cv=cv,
                                        scoring=make_scorer(roc_auc_score), verbose=3))

    res_gp_svm = gp_minimize(svm_objective, svm_params, n_calls=n_calls, random_state=1234, verbose=3, n_jobs=n_jobs)
    output.append([target_outcome, 'svm', 1-res_gp_svm.fun, res_gp_svm.x])

    ########################################################################################
#         print('#DT')
#         dt_model = DecisionTreeClassifier(class_weight='balanced', random_state=1234)
#         dt_params = [Categorical(["gini", "entropy", "log_loss"],name='criterion'),
#                      Categorical(['best', 'random'],name='splitter'),
#                      Integer(3, 10, "uniform", name='max_depth'),]

#         @use_named_args(dt_params)
#         def dt_objective(**params):
#             scoring = {
#                 'auc': make_scorer(roc_auc_score)
#                 }
#             dt_model.set_params(**params)

#             return 1 - np.mean(cross_val_score(dt_model, X, y, cv=cv,
#                                             scoring=make_scorer(roc_auc_score)))
#         res_gp_dt = gp_minimize(dt_objective, dt_params, n_calls=n_calls, random_state=1234, verbose=3, n_jobs=n_jobs)
#         output.append([target_outcome, 'dt', 1-res_gp_dt.fun, res_gp_dt.x])
    
# ##########################################################################################

#     print('#RF')
#     rf_model = RandomForestClassifier(class_weight='balanced', random_state=1234)
#     rf_params = [Categorical(["gini", "entropy", "log_loss"],name='criterion'),
#                  Integer(100, 500, "uniform", name='n_estimators'),
#                  Integer(3, 10, "uniform", name='max_depth'),]

#     @use_named_args(rf_params)
#     def rf_objective(**params):
#         scoring = {
#             'auc': make_scorer(roc_auc_score)
#             }
#         rf_model.set_params(**params)

#         return 1 - np.mean(cross_val_score(rf_model, X, y, cv=cv,
#                                         scoring=make_scorer(roc_auc_score)))
#     res_gp_rf = gp_minimize(rf_objective, rf_params, n_calls=n_calls, random_state=1234, verbose=3, n_jobs=n_jobs)
#     output.append([target_outcome, 'rf', 1-res_gp_rf.fun, res_gp_rf.x])
    
# ##########################################################################################

#     print('#XGB')
#     xgb_model = xgb.XGBClassifier(objective ='binary:logistic', tree_method='gpu_hist', gpu_id=0,  verbosity = 0,
#                                          importance_type = 'gain', scale_pos_weight = scale_pos_ratio, random_state=1234)
#     xgb_params = [Integer(100,500,"uniform", name='n_estimators'),
#                     Integer(3, 10, "uniform", name='max_depth'),
#                      Real(1e-5, 1e-1, 'log-uniform', name='learning_rate'),
#                      Real(1e-5, 1e-1, 'log-uniform', name='reg_alpha'),
#                      Real(1e-5, 1e-1, 'log-uniform', name='reg_lambda'),]
#     @use_named_args(xgb_params)
#     def xgb_objective(**params):
#         xgb_model.set_params(**params)

#         return 1 - np.mean(cross_val_score(xgb_model, X, y, cv=cv,
#                                         scoring=make_scorer(roc_auc_score)))
#     res_gp_xgb = gp_minimize(xgb_objective, xgb_params, n_calls=n_calls, random_state=1234, verbose=3, n_jobs=n_jobs)
#     output.append([target_outcome, 'xgb', 1-res_gp_xgb.fun, res_gp_xgb.x])

########################################################################################

######################################################################################################
outcome_combined_6months
#SVM
Iteration No: 1 started. Evaluating function at random point.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ................................ score: (test=0.624) total time=11.0min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 11.0min remaining:    0.0s


[CV] END ................................ score: (test=0.704) total time=12.4min


...

[CV] END ................................ score: (test=0.510) total time= 5.5min


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 10.5min remaining:    0.0s


[CV] END .................................. score: (test=nan) total time= 5.0min


/opt/conda/envs/rapids/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/envs/rapids/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/opt/conda/envs/rapids/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/opt/conda/envs/rapids/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/opt/conda/envs/rapids/lib/python3.10/site-packages/cuml/internals/api_decorators.py", line 188, in wrapper
    ret = func(*args, **kwargs)
  File "svc.pyx", line 617, in cuml.svm.svc.SVC.predict
  File "/opt/conda/envs/rapids/lib/pyt

ValueError: Input y contains NaN.

In [ ]:
pd.DataFrame(output, columns=['outcome', 'model', 'best_score', 'best_param']).to_csv('../Models/BS_result_svm6.csv', index = False, index_label=False)

In [ ]:
pd.DataFrame(output, columns=['outcome', 'model', 'best_score', 'best_param'])

In [ ]:
%%time

#GRID SEARCH
X = trainingData[features_columns]
# X = cudf.DataFrame(X)
outcomes = [
            'outcome_3months', 
            'outcome_combined_6months', 
            'outcome_combined_12months', 
            'outcome_combined_24months',
           ] 
model_names = ['LR', 'Lasso', 'ElasticNet', 'NB', 'SVM', 'DT', 'RF', 'XGB']


output = []
for outcome in outcomes:
    print(outcome)
    y = trainingData[outcome]
    scale_pos_ratio = y.value_counts()[0]/y.value_counts()[1]
    
    #MODELS
    lr_model = LogisticRegression(class_weight='balanced', random_state=1234)
    lasso_model = LogisticRegression(class_weight='balanced', penalty='l1', random_state=1234) #only the LIBLINEAR and SAGA (added in v0.19) solvers handle the L1 penalty
    elastic_model = LogisticRegression(class_weight='balanced', penalty = 'elasticnet', random_state=1234)
    gnb_model = GaussianNB()
    svc_model = SVC(class_weight='balanced', gamma = 10, kernel='rbf', cache_size=2000, random_state=1234)
    dt_model = DecisionTreeClassifier(class_weight='balanced', random_state=1234)
    rf_model = RandomForestClassifier(class_weight='balanced', random_state=1234)
    xgb_model = xgb.XGBClassifier(objective ='binary:logistic', tree_method='gpu_hist', gpu_id=0,  verbosity = 0,
                                     importance_type = 'gain', scale_pos_weight = scale_pos_ratio, random_state=1234)

    #PARAMS
    lr_params = {'solver': ['liblinear', 'newton-cholesky'],
                 'C': [0.1, 1.0, 10.0],
                 'max_iter': [80.0, 100.0, 120.0]}
    lasso_params = {'solver': ['saga', 'liblinear'],
                    'C': [0.1, 1, 10],
                    'max_iter': [80, 100, 120]},
    elastic_params = {'solver': ['saga', 'liblinear'],
                      'l1_ratio': [0.3, 0.5, 0.7],
                      'max_iter': [80, 100, 120]},
    gnb_params = {'var_smoothing': [1e-8, 5e-8, 1e-9, 5e-9]}
    svm_params={'C': [1.0, 10.0]}
    dt_params = {'criterion':["gini", "entropy", "log_loss"],
                 'splitter': ['best', 'random'],
                'max_depth': [3,5,7,9]}
    rf_params = {'criterion':["gini", "entropy", "log_loss"],
                 'n_estimators': [100, 200, 300],
                'max_depth': [3.0,5.0,7.0,9.0]}
    xgb_params = {'n_estimators': [100, 200, 300],
                'max_depth': [3.0,5.0,7.0,9.0],
                 'learning_rate': [3e-1, 3e-2, 3e-3],
                 'reg_alpha': [0.3, 0.5, 0.7],
                 'reg_lambda': [0.3, 0.5, 0.7],}

    #Models and params in DICT
    models_to_be_trained = [
        {'model_name': 'LR', 'model': lr_model, 'params': lr_params},
        {'model_name': 'Lasso', 'model': lasso_model, 'params': lasso_params},
        {'model_name': 'ElasticNet', 'model': elastic_model, 'params': elastic_params},
        {'model_name': 'NB', 'model': gnb_model, 'params': gnb_params},
        # {'model_name': 'SVM', 'model': svc_model, 'params': svm_params},
        {'model_name': 'DT', 'model': dt_model, 'params': dt_params},
        {'model_name': 'RF', 'model': rf_model, 'params': rf_params},
        {'model_name': 'XGB', 'model': xgb_model, 'params': xgb_params}
    ]
    
    #scoring
    # scoring = {
    #     'accuracy': make_scorer(balanced_accuracy_score),
    #     'sensitivity': make_scorer(recall_score),
    #     'specificity': make_scorer(recall_score,pos_label=0),
    #     'auc': make_scorer(roc_auc_score)
    #     }
    scoring = {
        'auc': make_scorer(roc_auc_score)
        }
    
    for item in models_to_be_trained:
        print(item['model_name'])
        gs = BayesSearchCV(item['model'],
                          search_spaces=item['params'],
                          scoring=make_scorer(roc_auc_score),
                           n_iter = 20,
                          cv=2,
                          verbose=3, 
                           n_jobs=5,
                           n_points=10,
                            random_state = 1234)
        gs.fit(X, y)
        output.append([outcome, item['model_name'], gs.best_params_])
        pickle.dump(gs.cv_results_, open('../Models/gs/' + outcome.split('_')[-1] + '_' + item['model_name'] + '.sav', 'wb'))


In [ ]:
%%time

X = trainingData[features_columns]
y = trainingData['outcome_combined_12months']
X = cudf.DataFrame(X)
# y = cudf.DataFrame(y)





In [ ]:
%%time

# svc_model = SVC(class_weight='balanced', kernel='poly', random_state=1234)
# svm_params={'C': [0.1, 1, 10], 'gamma': [1,10]}
# gs = GridSearchCV(svc_model,
#                   param_grid=svm_params,
#                   scoring=['average_precision', 'balanced_accuracy', 'roc_auc'],
#                   refit='roc_auc',
#                   cv=3,
#                   verbose=3,)
# gs.fit(X, y)

In [ ]:
%%time

svc_model = SVC(class_weight='balanced', C = 10, gamma= 10, kernel='rbf', cache_size= 2000, random_state=1234, verbose=3)
svc_model.fit(X, y)

In [ ]:
preds = svc_model.predict(X)

In [ ]:
preds

In [ ]:
classification_report(y, preds)

In [ ]:
output[5][1].keys()

In [ ]:
gs.best_score_

In [ ]:
output[5][1]['params'][output[5][1]['rank_test_balanced_accuracy'][0]]

In [ ]:
output[5][1]['params'][output[5][1]['rank_test_average_precision'][0]]

In [ ]:
pd.DataFrame(output, columns=['model', 'GS_result']).to_csv('../Models/GS_result.csv', index_label=False, index=False)

In [ ]:
item

In [ ]:
#Define number of split in k-fold

n_splits = 10

In [ ]:
#Create X set for model development

target_outcome = 'outcome_3months'
X = trainingData[features_columns]
y = trainingData[[target_outcome]]
print('X shape: ', X.shape)
print('y shape: ', y.shape)

#model parameters
params = {'xgb_lr': 0.6,
         'xgb_maxdepth': 7}

In [ ]:
%%time

#EXECUTE model training

kf = StratifiedKFold(n_splits=n_splits, random_state=1234, shuffle=True)
kf.get_n_splits(X)
models1 = pd.DataFrame(columns=['modelname', 'class_ratio'])
summary_result1 = []
cols = ['model_name', 'class_ratio', 'acc','spec','sens','auc', 'auprc', 'balance_accuracy', 'f1_score', 'ppv', 'npv']
split_counter = 0

#train model
for train_index, test_index in kf.split(X, y):
    #split data
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #Build models -> it can be commented if the models have been trained
    models_temp = pd.DataFrame(build_models(X_train, y_train[target_outcome], params, split_counter), columns=['modelname', 'class_ratio'])
    models1 = pd.concat([models1,models_temp]).reset_index(drop=True)
    split_counter+=1
        
#evaluate model
for modelname, classratio in models1.values:
    # print('======================================================================')
    print(modelname)
    model = pickle.load(open('./models/'+ target_outcome + '/'+ modelname + '.sav', 'rb'))
    summary_result1.append((str(model), classratio, ) + summariseResult (X_test, y_test[target_outcome], model) )       


summary_result1 = pd.DataFrame(summary_result1, columns=cols)
summary_result1['model_num'] = summary_result1.index



In [ ]:
print(target_outcome)
summary_result1['model_name'] = summary_result1.apply(lambda x: modelNameFixer(x.model_name), axis=1)
summary_result1.groupby('model_name').mean().sort_values(['auc'], ascending=False)

In [ ]:
summary_result1.to_csv("summaryResult_outcome1.csv")
summary_result1 = pd.read_csv("summaryResult_outcome1.csv")

bar = sns.catplot(x = "model_name",       # x variable name
            y = "auc",       # y variable name            
            data = summary_result1,     # dataframe to plot
            kind = "bar",
            height=5,
            aspect=5/2.5,
            ci = None)
ax = bar.facet_axis(0,0)
for p in ax.patches:
    ax.text(p.get_x() + 0.01, 
            p.get_height() * 1.01, 
            '{0:.4f}'.format(p.get_height()), 
            color='black', rotation='horizontal', fontsize=11)
    
# listOf_Yticks = np.arange(0.5, 0.7, 0.05)
ax.set_ylim(0.4, 1)
ax.set_ylabel('AUC Score', fontsize=11)
ax.set_xlabel('Method', fontsize=11)

In [ ]:
# kf = StratifiedKFold(n_splits=2, random_state=1234, shuffle=True)
# kf.get_n_splits(X)
# for train_index, test_index in kf.split(X, y):
#     #split data
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     trymodel = SVC(class_weight='balanced', C = 0.7, degree=2, kernel='poly', random_state=1234, cache_size=2048)
#     trymodel.fit(X_train,y_train)
#     print(summariseResult(X_test, y_test, trymodel))


In [ ]:
best_model1 = pickle.load(open('./models/outcome_3months/0DTModel.sav', 'rb'))

# pd.DataFrame([best_model3.feature_importances_], columns=X.columns).T.sort_values(0, ascending=False)
sorted_idx = best_model1.feature_importances_.argsort()
plt.figure(figsize=(5,7))
plt.barh(X.columns[sorted_idx][-10:], best_model1.feature_importances_[sorted_idx][-10:])
plt.xlabel("Decision Tree Feature Importance")
plt.show()

In [ ]:
best_model1 = pickle.load(open('./models/outcome_3months/0RFModel.sav', 'rb'))

# pd.DataFrame([best_model3.feature_importances_], columns=X.columns).T.sort_values(0, ascending=False)
sorted_idx = best_model1.feature_importances_.argsort()
plt.figure(figsize=(5,7))
plt.barh(X.columns[sorted_idx][-10:], best_model1.feature_importances_[sorted_idx][-10:])
plt.xlabel("Random Forest Feature Importance")
plt.show()

In [ ]:
best_model1 = pickle.load(open('./models/outcome_3months/0XGBoostModel.sav', 'rb'))

# pd.DataFrame([best_model3.feature_importances_], columns=X.columns).T.sort_values(0, ascending=False)
sorted_idx = best_model1.feature_importances_.argsort()
plt.figure(figsize=(5,7))
plt.barh(X.columns[sorted_idx][-10:], best_model1.feature_importances_[sorted_idx][-10:])
plt.xlabel("XGBoost Feature Importance")
plt.show()

# 6months

In [ ]:
target_outcome = 'outcome_combined_6months'
y = trainingData[[target_outcome]]

#model parameters
params = {'xgb_lr': 0.6,
         'xgb_maxdepth': 7}

In [ ]:
%%time

#EXECUTE model training

kf = StratifiedKFold(n_splits=n_splits, random_state=1234, shuffle=True)
kf.get_n_splits(X)
models2 = pd.DataFrame(columns=['modelname', 'class_ratio'])
summary_result2 = []
cols = ['model_name', 'class_ratio', 'acc','spec','sens','auc', 'auprc', 'balance_accuracy', 'f1_score', 'ppv', 'npv']
split_counter = 0

#train model
for train_index, test_index in kf.split(X, y):
    #split data
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #Build models -> it can be commented if the models have been trained
    models_temp = pd.DataFrame(build_models(X_train, y_train[target_outcome], params, split_counter), columns=['modelname', 'class_ratio'])
    models2 = pd.concat([models2,models_temp]).reset_index(drop=True)
    split_counter+=1
        
#evaluate model
for modelname, classratio in models2.values:
    # print('======================================================================')
    print(modelname)
    model = pickle.load(open('./models/'+ target_outcome + '/'+ modelname + '.sav', 'rb'))
    summary_result2.append((str(model), classratio, ) + summariseResult (X_test, y_test[target_outcome], model) )       


summary_result2 = pd.DataFrame(summary_result2, columns=cols)
summary_result2['model_num'] = summary_result2.index
# summary_result1['method_name'] = summary_result1.apply(lambda x: 'LR' if x.model_num%2 == 0 else 'XGBoost', axis=1)


In [ ]:
print(target_outcome)
summary_result2['model_name'] = summary_result2.apply(lambda x: modelNameFixer(x.model_name), axis=1)
summary_result2.groupby('model_name').mean().sort_values(['auc'], ascending=False)

In [ ]:
summary_result2.to_csv("summaryResult_outcome2.csv")
summary_result2 = pd.read_csv("summaryResult_outcome2.csv")

bar = sns.catplot(x = "model_name",       # x variable name
            y = "auc",       # y variable name            
            data = summary_result2,     # dataframe to plot
            kind = "bar",
            height=5,
            aspect=5/2.5,
            ci = None)
ax = bar.facet_axis(0,0)
for p in ax.patches:
    ax.text(p.get_x() + 0.01, 
            p.get_height() * 1.01, 
            '{0:.4f}'.format(p.get_height()), 
            color='black', rotation='horizontal', fontsize=11)
    
# listOf_Yticks = np.arange(0.5, 0.7, 0.05)
ax.set_ylim(0.4, 1)
ax.set_ylabel('AUC Score', fontsize=11)
ax.set_xlabel('Method', fontsize=11)

In [ ]:
best_model2 = pickle.load(open('./models/outcome_combined_6months/0DTModel.sav', 'rb'))

# pd.DataFrame([best_model3.feature_importances_], columns=X.columns).T.sort_values(0, ascending=False)
sorted_idx = best_model2.feature_importances_.argsort()
plt.figure(figsize=(5,7))
plt.barh(X.columns[sorted_idx][-10:], best_model2.feature_importances_[sorted_idx][-10:])
plt.xlabel("Decision Tree Feature Importance")
plt.show()

In [ ]:
best_model2 = pickle.load(open('./models/outcome_combined_6months/0RFModel.sav', 'rb'))

# pd.DataFrame([best_model3.feature_importances_], columns=X.columns).T.sort_values(0, ascending=False)
sorted_idx = best_model2.feature_importances_.argsort()
plt.figure(figsize=(5,7))
plt.barh(X.columns[sorted_idx][-10:], best_model2.feature_importances_[sorted_idx][-10:])
plt.xlabel("Random Forest Feature Importance")
plt.show()

In [ ]:
best_model2 = pickle.load(open('./models/outcome_combined_6months/0XGBoostModel.sav', 'rb'))

# pd.DataFrame([best_model3.feature_importances_], columns=X.columns).T.sort_values(0, ascending=False)
sorted_idx = best_model2.feature_importances_.argsort()
plt.figure(figsize=(5,7))
plt.barh(X.columns[sorted_idx][-10:], best_model2.feature_importances_[sorted_idx][-10:])
plt.xlabel("XGBoost Feature Importance")
plt.show()

# 12 months

In [ ]:
target_outcome = 'outcome_combined_12months'
y = trainingData[[target_outcome]]

#model parameters
params = {'xgb_lr': 0.6,
         'xgb_maxdepth': 10}

In [ ]:
%%time

#EXECUTE model training

kf = StratifiedKFold(n_splits=n_splits, random_state=1234, shuffle=True)
kf.get_n_splits(X)
models3 = pd.DataFrame(columns=['modelname', 'class_ratio'])
summary_result3 = []
cols = ['model_name', 'class_ratio', 'acc','spec','sens','auc', 'auprc', 'balance_accuracy', 'f1_score', 'ppv', 'npv']
split_counter = 0

#train model
for train_index, test_index in kf.split(X, y):
    #split data
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #Build models -> it can be commented if the models have been trained
    models_temp = pd.DataFrame(build_models(X_train, y_train[target_outcome], params, split_counter), columns=['modelname', 'class_ratio'])
    models3 = pd.concat([models3,models_temp]).reset_index(drop=True)
    split_counter+=1
        
#evaluate model
for modelname, classratio in models3.values:
    # print('======================================================================')
    print(modelname)
    model = pickle.load(open('./models/'+ target_outcome + '/'+ modelname + '.sav', 'rb'))
    summary_result3.append((str(model), classratio, ) + summariseResult (X_test, y_test[target_outcome], model) )       


summary_result3 = pd.DataFrame(summary_result3, columns=cols)
summary_result3['model_num'] = summary_result3.index
# summary_result1['method_name'] = summary_result1.apply(lambda x: 'LR' if x.model_num%2 == 0 else 'XGBoost', axis=1)


In [ ]:
print(target_outcome)
summary_result3['model_name'] = summary_result3.apply(lambda x: modelNameFixer(x.model_name), axis=1)
summary_result3.groupby('model_name').mean().sort_values(['auc'], ascending=False)

In [ ]:
summary_result3.to_csv("summaryResult_outcome3.csv")
summary_result3 = pd.read_csv("summaryResult_outcome3.csv")

bar = sns.catplot(x = "model_name",       # x variable name
            y = "auc",       # y variable name            
            data = summary_result3,     # dataframe to plot
            kind = "bar",
            height=5,
            aspect=5/2.5,
            ci = None)
ax = bar.facet_axis(0,0)
for p in ax.patches:
    ax.text(p.get_x() + 0.01, 
            p.get_height() * 1.01, 
            '{0:.4f}'.format(p.get_height()), 
            color='black', rotation='horizontal', fontsize=11)
    
# listOf_Yticks = np.arange(0.5, 0.7, 0.05)
ax.set_ylim(0.4, 1)
ax.set_ylabel('AUC Score', fontsize=11)
ax.set_xlabel('Method', fontsize=11)

In [ ]:
best_model3 = pickle.load(open('./models/outcome_combined_12months/0DTModel.sav', 'rb'))

# pd.DataFrame([best_model3.feature_importances_], columns=X.columns).T.sort_values(0, ascending=False)
sorted_idx = best_model3.feature_importances_.argsort()
plt.figure(figsize=(5,7))
plt.barh(X.columns[sorted_idx][-10:], best_model3.feature_importances_[sorted_idx][-10:])
plt.xlabel("Decision Tree Feature Importance")
plt.show()

In [ ]:
best_model3 = pickle.load(open('./models/outcome_combined_12months/0RFModel.sav', 'rb'))

# pd.DataFrame([best_model3.feature_importances_], columns=X.columns).T.sort_values(0, ascending=False)
sorted_idx = best_model3.feature_importances_.argsort()
plt.figure(figsize=(5,7))
plt.barh(X.columns[sorted_idx][-10:], best_model3.feature_importances_[sorted_idx][-10:])
plt.xlabel("Random Forest Feature Importance")
plt.show()

In [ ]:
best_model3 = pickle.load(open('./models/outcome_combined_12months/0XGBoostModel.sav', 'rb'))

# pd.DataFrame([best_model3.feature_importances_], columns=X.columns).T.sort_values(0, ascending=False)
sorted_idx = best_model3.feature_importances_.argsort()
plt.figure(figsize=(5,7))
plt.barh(X.columns[sorted_idx][-10:], best_model3.feature_importances_[sorted_idx][-10:])
plt.xlabel("XGBoost Feature Importance")
plt.show()